# Update on yesterday progress

In [83]:
import requests
import numpy as np
import pandas as pd

disc_genes="T:\colin\BRice\V2\gwas-gene-discovery\MLM.DTF.GWAS.Results\summary_genes_discovered.txt"
lists="T:\colin\BRice\V2\gwas-gene-discovery\mock_keyword_list.txt"

In [88]:
def knetapi(disc_genes):
    '''Use the genes from the summary file to search for Knetminer genome api.'''
    with open(disc_genes, "r") as fs:
        next(fs)
        genes = []
        with open(lists, "r") as fl:
            pheno = []
            for line in fl:
                pheno.append(line.rstrip())
            #print("The phenotype of interests are {}".format(pheno))
            for line in fs:
                col = line.split("\t")
                genes.append(col[0])
            genelist = (",").join(genes) #join all iterative elements by ,
            #print(genelist)

            #use str.join() to convert multiple elments in a list into one string.
            keyw1 = "%20OR%20".join("({})".format(i.replace(" ", "+AND+")) for i in pheno)
            link = "http://knetminer.rothamsted.ac.uk/riceknet/genome?"
            parameters = {"keyword":keyw1, "list":genelist}
            r = requests.get(link, params=parameters)

            #check if request is succesfsul.
            if not r.ok:
                r.raise_for_status()
                sys.exit()

            #printing out genetable to a file.
            decoded=r.json()[u'geneTable'].split("\t")
            #remove space or newline at the end.
            decoded=(decoded)[:-1]
            
            ncolumns=9
            #tabulate the list to be any dimension as long as it's dividable by 9 columns.
            genetable=np.array(decoded).reshape(len(decoded)/ncolumns, ncolumns)
            df = pd.DataFrame(genetable[1:,:], columns=genetable[0,:])
                
            knetgenes=df[u'ACCESSION']
            knetscore=df[u'SCORE']
            genechr=df[u'CHRO']
            start=df[u'START']
            
            '''
            #select the rows we want
            trimmed = df.iloc[:, [1,3,4,6]]
            
            return trimmed
            '''
            
            #alternatively delete the ones we don't want.
            df = df.drop([u'ONDEX-ID', u'GENE NAME', u'TAXID', u'USER', u'QTL'], axis=1)
            
            #rename column name using df.rename and a dictionary, inplace.
            df.rename (
            columns={
                "ACCESSION":"KNETGENES",
                "CHRO":"GeneCHR",
                "START":"GeneStart",
                "SCORE":"KnetScore"
            }, inplace=True
            )
            
            #return df
        
        
            content = pd.read_csv(summary, sep="\t")
            content['Knetscore'] = knetscore
            content['gene_start'] = start           
            return content
             # define values needed to generate network view url
            keyw2 = "+OR+".join("({})".format(i.replace(" ", "+AND+")) for i in pheno)
            
            #iterate through every Knetgene to construct url for each
            link=[]
            for i in genes:
                address="http://knetminer.rothamsted.ac.uk/{}/genepage?list={}&keyword={}".format("riceknet", i, keyw2)
                r=requests.get(address)
                link.append(r.url)
                
            #Theoretically all you need to do at this point is to append df (without KNETGENES) and link to summary
            #add link list as a new column of df.
            df['network_view'] = link
            #return df
            #print(len(link))
            #return df
knetapi(disc_genes)            

,GENE,CHROMOSOME,SNP_number,SNP_Base_Position,p-value,logP,SNP_effect,allele_info,Gene_description,strand,Knetscore,gene_start
0,Os01g0607900,1,36547,23977924,1.859152e-07,6.730685,intron_variant,"[u'T', u'C']",None,None,125.60,1270320
1,Os01g0607900,1,36553,23978792,7.915247e-07,6.101536,intron_variant,"[u'A', u'G']",None,None,10.64,2898787
2,Os11g0526200,11,172992,19095542,1.975935e-07,6.704227,downstream_gene_variant,"[u'A', u'C']",None,None,10.64,2893694
3,Os02g0117500,2,354514,918560,7.427877e-07,6.129135,upstream_gene_variant,"[u'C', u'T']",None,None,5.94,3286611
4,Os05g0185700,5,542826,5261367,4.509205e-07,6.345900,intron_variant,"[u'C', u'T']",None,None,0.00,3322909
5,Os05g0186900,5,542983,5329395,1.194229e-08,7.922912,upstream_gene_variant,"[u'G', u'T']",None,None,NaN,NaN
6,Os05g0186900,5,542985,5330001,1.403204e-10,9.852879,5_prime_UTR_variant,"[u'T', u'C']",None,None,NaN,NaN
7,Os05g0186900,5,542987,5331815,2.573877e-09,8.589412,upstream_gene_variant,"[u'G', u'A']",None,None,NaN,NaN
8,Os05g0187000,5,542991,5333585,1.065410e-11,10.972483,upstream_gene_variant,"[u'A', u'G']",None,None,NaN,NaN
9,Os05g0187000,5,542994,5335283,6.466251e-07,6.189347,intron_variant,"[u'A', u'G']",None,None,NaN,NaN


# How to open text file as dataframe in pandas

In [59]:
summary="T:\colin\BRice\V2\gwas-gene-discovery\MLM.blupWidth.GWAS.Results\summary_genes_discovered.txt"

In [60]:
import pandas as pd
import numpy as np

In [61]:
content = pd.read_csv(summary, sep="\t")
content.head()

,GENE,CHR,SNPnum,snpBP,P,logP,SNPeff,allele,designation,strand
0,Os01g0607900,1,36547,23977924,1.859152e-07,6.730685,intron_variant,"[u'T', u'C']",None,None
1,Os01g0607900,1,36553,23978792,7.915247e-07,6.101536,intron_variant,"[u'A', u'G']",None,None
2,Os11g0526200,11,172992,19095542,1.975935e-07,6.704227,downstream_gene_variant,"[u'A', u'C']",None,None
3,Os02g0117500,2,354514,918560,7.427877e-07,6.129135,upstream_gene_variant,"[u'C', u'T']",None,None
4,Os05g0185700,5,542826,5261367,4.509205e-07,6.345900,intron_variant,"[u'C', u'T']",None,None


# Final version